# BioMek CSV ProtoCompiler

An Architectural Shift in the Biomek CSV Compiler

In [ ]:
cd ../

In [1]:
import pandas as pd
from IPython.display import display
import matplotlib.pyplot as plt
import numpy as np

#Well Volume (Property of the Plate)
#Might Want a Plate Class Downstream
min_volume = 5     #uL
max_volume = 1200  #uL

#Transfer Volume (Property of the Pipette)
#Might want to make a Pipette Class
min_transfer = 5   #uL
max_transfer = 180 #uL

#Plate Limits
#This should probably be cleaned up
dead_volume = 50 #uL
safety_factor = 2 
DILUTION_SAFETY_FACTOR = 2
ideal_transfer_volume = 8 #uL
dilution_volume = ideal_transfer_volume


#deck_file = 'Deck_Setup.csv'
deck_file = '5WellTest.csv'
deck_file = 'data/initial_comb_media_deck.csv'
deck_file = 'data/indigoidine_media.csv'

In [2]:
def concentration_to_moles(df):
    #Convert NaNs to Zeros
    df = df.fillna(value=0)
    
    #Convert Molar Concentrion to Moles
    for column in df.loc[:,~df.columns.isin(['Volume','Target'])].columns:
        df.loc[:,column] = df[column]*df['Volume']*1e-6
    return df


class BioMek(object):
    
    def __init__(self,deck_df):
        self.deck_df = deck_df.loc[~deck_df['Target'],deck_df.columns != 'Target']
        self.goal_df = deck_df.loc[ deck_df['Target'],deck_df.columns != 'Target']
        #self.water_wells =
        
        transfer_columns = ['srcpos','srcwell','destpos','destwell','vol']
        self.transfer_df = pd.DataFrame(columns=transfer_columns)
        
        
    def transfer(self, source_plate, source_well, dest_plate, dest_well, transfer_volume):
        total_transfered = 0 #uL
        
        #Check to see if there is enough volume for the transfer!
        if transfer_volume > (self.deck_df.loc[(source_plate,source_well)]['Volume'] + dead_volume):
            raise('Transfer Pulling Too Much Volume! {}'.format(transfer_volume))
        
        
        #Add Transfer To Ledger
        while total_transfered < transfer_volume:
            if transfer_volume - total_transfered > max_transfer:
                values = [source_plate,source_well,dest_plate,dest_well,max_transfer]
                total_transfered += max_transfer
            else:
                values = [source_plate,source_well,dest_plate,dest_well,transfer_volume-total_transfered]
                total_transfered += transfer_volume-total_transfered
            
            self.transfer_df = self.transfer_df.append(dict(zip(self.transfer_df.columns,values)),ignore_index=True)
            
        
        #Update Deck State
        source_composition = self.deck_df.xs((source_plate,source_well))
        transfer = source_composition*(transfer_volume/source_composition['Volume'])

        self.deck_df.loc[(source_plate,source_well)] -= transfer
        try:
            self.deck_df.loc[(dest_plate  ,dest_well  )] += transfer
        except:
            columns = self.deck_df.reset_index().columns
            data = [dest_plate,dest_well] + [0]*(len(columns) - 2)
            new_well = pd.DataFrame(dict(zip(columns,data)),index=[0]).set_index(['Plate','Well']) + transfer
            self.deck_df = self.deck_df.append(new_well)
            
    
    def get_well_state(self,plate,well):
        
        try:
            return self.deck_df.loc[(dest_plate,dest_well)]
        except:
            columns = self.deck_df.reset_index().columns
            data = [dest_plate,dest_well] + [0]*(len(columns) - 2)
            new_well = pd.DataFrame(dict(zip(columns,data)),index=[0]).set_index(['Plate','Well'])
            self.deck_df = self.deck_df.append(new_well)
        return self.deck_df.loc[(dest_plate,dest_well)]
            
    
    def transfer_water(self,dest_plate,dest_well,transfer_volume):
        total_transfered = 0
        
        while total_transfered < transfer_volume:
            if transfer_volume - total_transfered > max_transfer:
                water_plate, water_well = self.find_water(max_transfer)
                self.transfer(water_plate,water_well,dest_plate,dest_well,max_transfer)
                total_transfered += max_transfer
            else:
                water_plate, water_well = self.find_water(transfer_volume - total_transfered)
                self.transfer(water_plate,water_well,dest_plate,dest_well,transfer_volume - total_transfered)
                total_transfered += transfer_volume - total_transfered
                
            
    def dilute(self,reagent_wells,solute,moles_needed,transfer_volume=ideal_transfer_volume):
        '''Creates a Diluted Version of the Source Plate'''
        
        #Find The Right Well
        ENOUGH_VOLUME = reagent_wells['Volume'] > min_volume * safety_factor + transfer_volume
        reagent_wells = reagent_wells.loc[ENOUGH_VOLUME]
        reagent_wells['dilution_factor'] = reagent_wells[solute]/reagent_wells['Volume']

        #Calculate Required transfer volume
        #source = reagent_wells['dilution_factor'].idxmin()
        #source = self.deck_df.loc[(source_plate,source_well)]
        #source_well_moles = source[solute]
        
        reagent_wells['dilution_volume'] = moles_needed*max_volume*reagent_wells['Volume']/(transfer_volume*reagent_wells[solute])
        #display(reagent_wells)
        reagent_wells = reagent_wells.loc[reagent_wells['dilution_volume'] < (reagent_wells['Volume'] - dead_volume)]
        #display('after',reagent_wells)
        
        if len(reagent_wells) == 0:
            reagent=solute
            print(solute)
            REAGENT_WELL = (self.deck_df.loc[:,~biomek.deck_df.columns.isin([reagent,'Volume'])] == 0).all(1)
            CONTAINS_REAGENT = (biomek.deck_df.loc[:,biomek.deck_df.columns == reagent] > 0).any(1)
            reagent_wells = self.deck_df.loc[REAGENT_WELL & CONTAINS_REAGENT]
            display(reagent_wells)
            raise('Not Enough {}! Add More To Reagent Plate.'.format(solute))        
        
        
        source = reagent_wells['dilution_factor'].idxmin()
        dilution_volume = max(min_volume,reagent_wells.loc[source]['dilution_volume'])
        
        #display(reagent_wells)
        #print('Dilution Performed!, Volume: {}'.format(dilution_volume))
        
        #Check to see if dilution volume is above minimum volume
        #if dilution_volume < min_volume:
        #    dilution_volume = min_volume
        
        
        
        
        #Transfer into New Well
        dilution_plate, dilution_well = self.allocate_well()
        self.transfer(*source,dilution_plate,dilution_well,dilution_volume)
        
        #Fill With Water
        self.transfer_water(dilution_plate,dilution_well,max_volume - dilution_volume)
        

    def allocate_well(self):
        '''Allocate a New Well'''
        plate = 'mixing_plate'
        try:
            well = len(self.deck_df.loc[plate])+1
        except:
            well = 1
            
        if well > 96:
            display(biomek.deck_df)
            display(self.transfer_df)
            raise('Too Many Wells: Implement New Plate Method')
        return (plate,well)
    
    def find_water(self,transfer_volume):
        WATER_WELL = (biomek.deck_df.loc[:,~biomek.deck_df.columns.isin(['Volume'])] == 0).all(1) & (biomek.deck_df.loc[:,biomek.deck_df.columns == 'Volume'] > (transfer_volume+dead_volume)).any(1)
        source = biomek.deck_df[WATER_WELL].iloc[0]
        return source.name

In [3]:
deck_df = pd.read_csv(deck_file)
deck_df = deck_df.set_index(['Plate','Well'])
deck_df = concentration_to_moles(deck_df)

biomek = BioMek(deck_df)

#display(biomek.deck_df)
#display(biomek.goal_df)
#display(biomek.transfer_df)

In [4]:
#Iterate Through Destination Wells
for (dest_plate,dest_well),solution in biomek.goal_df.iterrows():
    print(dest_plate,dest_well)
    
    #Find Solute & Moles Needed
    for i,(reagent,moles) in enumerate(solution.loc[solution.index != 'Volume'].iteritems()):
        #print(reagent,moles)
        while moles > 0:
            
            #Get All Reagent Wells
            REAGENT_WELL = (biomek.deck_df.loc[:,~biomek.deck_df.columns.isin([reagent,'Volume'])] == 0).all(1)
            CONTAINS_REAGENT = (biomek.deck_df.loc[:,biomek.deck_df.columns == reagent] > 0).any(1)
            reagent_wells = biomek.deck_df.loc[REAGENT_WELL & CONTAINS_REAGENT]

            #See There Are Enough Moles in The Reagent Wells ON Deck from any Well
            
            #This line is the problem...
            #reagent_wells[reagent] = ((reagent_wells['Volume'] - dead_volume)/reagent_wells['Volume']) * reagent_wells[reagent]
            
            reagent_wells = reagent_wells.loc[reagent_wells[reagent] > moles]
            
            
            if len(reagent_wells):
                #print('Enough Moles of {} In Atleast One Well!'.format(reagent))
                #Find Wells require above the minimum pipette volume
                reagent_wells['volume_needed'] = reagent_wells['Volume']*(moles/reagent_wells[reagent])
                ENOUGH_VOLUME = (reagent_wells['Volume'] - dead_volume > reagent_wells['volume_needed'])
                source_wells = reagent_wells.loc[(reagent_wells['volume_needed'] > min_volume) & (reagent_wells[reagent] > moles) & ENOUGH_VOLUME]
                
                #Find Wells With Enough Volume For Transfer
                #source_wells['transfer_volume'] = (source_wells['Volume']*(moles/source_wells[reagent]))
                source_wells = source_wells.loc[source_wells['volume_needed'] < (source_wells['Volume'] - dead_volume)]
                #display(source_wells)
                
                if len(source_wells):
                    
                    #Get Least Dilute Well
                    source = source_wells['volume_needed'].idxmin()
                    transfer_volume = source_wells.loc[source]['volume_needed']
                    #print('Transfering {}uL {}'.format(transfer_volume,reagent))
                    #Perform Transfer
                    biomek.transfer(*source,dest_plate,dest_well,transfer_volume)

                    break
                    
                else:
                    biomek.dilute(reagent_wells,reagent,moles,transfer_volume=ideal_transfer_volume)
                    


                    
            else:
                print(solute,moles)
                display(reagent_wells)
                display(biomek.transfer_df)
                display(biomek.deck_df.loc['src_plate'])
                raise('No Valid Well')
    
    #Fill Remaining Volume with Water
    transfer_volume = biomek.goal_df.loc[(dest_plate,dest_well)]['Volume'] - biomek.deck_df.loc[(dest_plate,dest_well)]['Volume']
    biomek.transfer_water(dest_plate,dest_well,transfer_volume)



dest_plate 1
dest_plate 9
dest_plate 17
dest_plate 2
dest_plate 10
dest_plate 18
dest_plate 3
dest_plate 11
dest_plate 19
dest_plate 4
dest_plate 12
dest_plate 20
dest_plate 5
dest_plate 13
dest_plate 21
dest_plate 6
dest_plate 14
dest_plate 22
dest_plate 7
dest_plate 15
dest_plate 23
dest_plate 8
dest_plate 16
dest_plate 24
dest_plate 25
dest_plate 33
dest_plate 41
dest_plate 26
dest_plate 34
dest_plate 42
dest_plate 27
dest_plate 35
dest_plate 43
dest_plate 28
dest_plate 36
dest_plate 44
dest_plate 29
dest_plate 37
dest_plate 45
dest_plate 30
dest_plate 38
dest_plate 46
dest_plate 31
dest_plate 39
dest_plate 47


#Order Solutions from Least Dilution Required to Most to Account for large volumes...
#display(biomek.deck_df.loc[:,'Volume'][('dest_plate',1)])
#DECK_WELL = lambda x,y: (biomek.deck_df['Plate']==x)&(biomek.deck_df['Well']==y)

#For Each Target Well 
for (dest_plate,dest_well),solution in biomek.goal_df.iterrows():
    num_solutions = len(solution)-1
    print(dest_plate,dest_well)
    
    #Find a Well 
    for i,(solute,moles) in enumerate(solution.loc[solution.index != 'Volume'].iteritems()):
        #solutions_left = num_solutions - i
        #volume_left = biomek.get_well_state(dest_plate,dest_well)['Volume']
        #print(solute,moles)
        #print(solutions_left,volume_left)
        #print('')
        
        if moles > 0:
            while True:
                #print(dest_plate,dest_well,solute,moles)

                #Find a Well containg enough available moles
                REAGENT_WELL = (biomek.deck_df.loc[:,~biomek.deck_df.columns.isin([solute,'Volume'])] == 0).all(1)
                CONTAINS_SOLUTE = (biomek.deck_df.loc[:,biomek.deck_df.columns == solute] > 0).any(1)
                available_moles = biomek.deck_df.loc[REAGENT_WELL & CONTAINS_SOLUTE]
                available_moles[solute] = ((available_moles['Volume'] - dead_volume)/available_moles['Volume']) * available_moles[solute]
                source = available_moles.loc[available_moles[solute]>moles]
                
                #Make Sure Well Is Concentrated Enough
                #display(source['Volume']*(moles/source[solute]))
                
                #display(source)
                #raise('error')
                
                #Check to See that Wells are Concentrated Enough to hit Target Size
                #SOLUTE_WELL = (biomek.deck_df.loc[:,biomek.deck_df.columns == solute] > moles).any(1) & (biomek.deck_df.loc[:,~biomek.deck_df.columns.isin([solute,'Volume'])] == 0).all(1)
                #source   = biomek.deck_df.loc[SOLUTE_WELL]

                if len(source) == 0:
                    #Is there a Usable Well?
                    display(biomek.transfer_df)
                    display(biomek.deck_df.loc['src_plate'])
                    print(solute,moles)
                    raise('No Valid Well')
                #elif len(source) == 1:
                #    source = source.iloc[0]

                #elif len(source) != 1:
                #    #Pick Smallest Valid Well
                #    source = source.sort_values(solute).iloc[0]

                #display(source)
                #display(biomek.deck_df)
                source_plate,source_well = source.name
                #source_plate = source.reset_index()['Plate'].values[0]
                #source_well  = source.reset_index()['Well'].values[0]
                transfer_volume = (source['Volume']*(moles/source[solute]))#.values[0]

                #Check that the volume is in the right range
                if transfer_volume < min_volume*safety_factor:
                    #create a new diluted well that will be closer to the right value
                    biomek.dilute(source_plate,source_well,dest_plate,dest_well,solute)
                else:
                    #do the transfer
                    if solute == 'Glucose':
                        print('Transfering Glucose Volume: {:0.2f}uL'.format(transfer_volume))
                    biomek.transfer(source_plate,source_well,dest_plate,dest_well,transfer_volume)
                    break
        
    print('Volume Before Fill {:0.2f}'.format(biomek.get_well_state(dest_plate,dest_well)['Volume']))
    
    #Fill Well to Required Volume!
    transfer_volume = biomek.goal_df.loc[(dest_plate,dest_well)]['Volume'] - biomek.deck_df.loc[(dest_plate,dest_well)]['Volume']
    #print(transfer_volume)
    biomek.transfer_water(dest_plate,dest_well,transfer_volume)
    #water_plate,water_well = biomek.find_water(transfer_volume)
    #WATER_WELL = (biomek.deck_df.loc[:,~biomek.deck_df.columns.isin(['Volume'])] == 0).all(1) & (biomek.deck_df.loc[:,biomek.deck_df.columns == 'Volume'] > transfer_volume).any(1)
    #source = biomek.deck_df[WATER_WELL]
    #source_plate = source.reset_index()['Plate'].values[0]
    #source_well  = source.reset_index()['Well'].values[0]
    #biomek.transfer(water_plate,water_well,dest_plate,dest_well,transfer_volume)
    

In [5]:
display(biomek.transfer_df)
display(biomek.deck_df)

,srcpos,srcwell,destpos,destwell,vol
0,src_plate,17,dest_plate,1,43.970681
1,src_plate,1,dest_plate,1,40.685574
2,src_plate,5,dest_plate,1,24.662641
3,src_plate,3,dest_plate,1,11.077191
4,src_plate,8,dest_plate,1,6.002516
5,src_plate,13,mixing_plate,1,180.000000
6,src_plate,13,mixing_plate,1,16.612690
7,water_plate,1,mixing_plate,1,180.000000
8,water_plate,1,mixing_plate,1,180.000000
9,water_plate,1,mixing_plate,1,180.000000


Glucose   Na2HPO4      NaCl    KH2PO4  (NH4)2SO4  \
Plate        Well                                                      
src_plate    1     0.000000  0.057509  0.000000  0.000000   0.000000   
             2     0.000000  0.446864  0.000000  0.000000   0.000000   
             3     0.000000  0.000000  0.000000  0.275878   0.000000   
             4     0.000000  0.000000  0.000000  0.481537   0.000000   
             5     0.000000  0.000000  0.499081  0.000000   0.000000   
             6     0.000000  0.000000  0.058070  0.000000   0.000000   
             7     0.000000  0.000000  0.737298  0.000000   0.000000   
             8     0.000000  0.000000  0.000000  0.000000   0.143772   
             9     0.000000  0.000000  0.000000  0.000000   0.000000   
             10    0.000000  0.000000  0.000000  0.000000   0.000000   
             11    0.000000  0.000000  0.000000  0.000000   0.000000   
             12    0.000000  0.000000  0.000000  0.000000   0.000000   
             13    0.000000  0.000000  0.000000  0.000000   0.000000   
             14    0.000000  0.000000  0.000000  0.000000   0.000000   
             15    0.000000  0.000000  0.000000  0.000000   0.000000   
             16    0.000000  0.000000  0.000000  0.000000   0.000000   
             17    0.002089  0.000000  0.000000  0.000000   0.000000   
             18    0.014068  0.000000  0.000000  0.000000   0.000000   
             19    0.029144  0.000000  0.000000  0.000000   0.000000   
             20    0.034994  0.000000  0.000000  0.000000   0.000000   
water_plate  1     0.000000  0.000000  0.000000  0.000000   0.000000   
             2     0.000000  0.000000  0.000000  0.000000   0.000000   
             3     0.000000  0.000000  0.000000  0.000000   0.000000   
             4     0.000000  0.000000  0.000000  0.000000   0.000000   
             5     0.000000  0.000000  0.000000  0.000000   0.000000   
             6     0.000000  0.000000  0.000000  0.000000   0.000000   
             7     0.000000  0.000000  0.000000  0.000000   0.000000   
             8     0.000000  0.000000  0.000000  0.000000   0.000000   
             9     0.000000  0.000000  0.000000  0.000000   0.000000   
             10    0.000000  0.000000  0.000000  0.000000   0.000000   
...                     ...       ...       ...       ...        ...   
dest_plate   24    0.002163  0.051383  0.020137  0.022598   0.003781   
             25    0.001899  0.019364  0.030010  0.027740   0.014561   
mixing_plate 30    0.000000  0.000000  0.000000  0.000000   0.000000   
dest_plate   33    0.001899  0.019364  0.030010  0.027740   0.014561   
             41    0.001899  0.019364  0.030010  0.027740   0.014561   
mixing_plate 31    0.000000  0.000000  0.000000  0.000000   0.000000   
dest_plate   26    0.002675  0.036007  0.018107  0.025395   0.019973   
mixing_plate 32    0.000000  0.000000  0.000000  0.000000   0.000000   
             33    0.000000  0.000000  0.000000  0.000000   0.000000   
dest_plate   34    0.002675  0.036007  0.018107  0.025395   0.019973   
             42    0.002675  0.036007  0.018107  0.025395   0.019973   
             27    0.000768  0.028052  0.054650  0.007112   0.021387   
             35    0.000768  0.028052  0.054650  0.007112   0.021387   
             43    0.000768  0.028052  0.054650  0.007112   0.021387   
             28    0.002373  0.044572  0.046306  0.030686   0.018034   
             36    0.002373  0.044572  0.046306  0.030686   0.018034   
             44    0.002373  0.044572  0.046306  0.030686   0.018034   
             29    0.002843  0.047972  0.039090  0.019955   0.008963   
             37    0.002843  0.047972  0.039090  0.019955   0.008963   
             45    0.002843  0.047972  0.039090  0.019955   0.008963   
             30    0.000340  0.023730  0.003011  0.018481   0.010376   
mixing_plate 34    0.000000  0.000000  0.442562  0.000000   0.000000   
dest_plate   38    0.000340  0.023730  0.003011  0.018481   

In [6]:
#Generate 4 CSVs for BIOMEK
water_mix_df  = biomek.transfer_df.loc[(biomek.transfer_df['destpos']=='mixing_plate') & (biomek.transfer_df['srcpos']=='water_plate')]
water_dest_df = biomek.transfer_df.loc[(biomek.transfer_df['destpos']=='dest_plate') & (biomek.transfer_df['srcpos']=='water_plate')]
mix_df =   biomek.transfer_df.loc[(biomek.transfer_df['destpos']=='mixing_plate') & (biomek.transfer_df['srcpos']!='water_plate')]
src_df = biomek.transfer_df.loc[(biomek.transfer_df['destpos']=='dest_plate') & (biomek.transfer_df['srcpos']=='src_plate')] 
dest_df =  biomek.transfer_df.loc[(biomek.transfer_df['destpos']=='dest_plate') & (biomek.transfer_df['srcpos']=='mixing_plate')]
#dest_df =  biomek.transfer_df.loc[(biomek.transfer_df['destpos']=='dest_plate')]

print(sum(dest_df['vol']))
#display(dest_df)

#line terminator required for csvs to be read by biomek software properly...
water_mix_df.to_csv('biomek_files/water_mix.csv',index=False,line_terminator='\r\n')
water_dest_df.to_csv('biomek_files/water_dest.csv',index=False,line_terminator='\r\n')
mix_df.to_csv('biomek_files/mix.csv',index=False,line_terminator='\r\n')
src_df.to_csv('biomek_files/src.csv',index=False,line_terminator='\r\n')
dest_df.to_csv('biomek_files/dest.csv',index=False,line_terminator='\r\n')


SOURCE = lambda x,y: (biomek.transfer_df['srcpos'] == x) & (biomek.transfer_df['srcwell'] == y)
DEST = lambda x,y: (biomek.transfer_df['destpos'] == x) & (biomek.transfer_df['destwell'] == y)
#display(biomek.transfer_df.loc[DEST('dest_plate')].sort_values('srcwell',ascending=False).set_index(['srcpos','srcwell','destpos','destwell']))

print(len(biomek.deck_df.loc['dest_plate']))
display(biomek.deck_df.loc['dest_plate'])
#display(water_mix_df)
#display(water_dest_df.sort_values('vol'))

#display(biomek.deck_df.loc['mixing_plate'])
dep_well_df = biomek.transfer_df.loc[DEST('mixing_plate',34)] 
well_df = biomek.transfer_df.loc[SOURCE('mixing_plate',34)]
total_df = biomek.transfer_df.loc[SOURCE('mixing_plate',33) | DEST('mixing_plate',33)]
sum(dep_well_df['vol']) - sum(well_df['vol'])

def well_change(well,row):
    if (row['srcpos'] == well[0]) & (row['srcwell'] == well[1]):
        return -1*row['vol']
    else:
        return row['vol']


#total_df['vol change'] = total_df.apply(lambda x: well_change(('mixing_plate',33),x),axis=1)
display(total_df.sort_values('destwell',ascending=False))

plt.figure()
#plt.plot(np.cumsum(total_df.sort_values('destwell')['vol change']))
#print(np.cumsum(total_df.sort_values('destwell',ascending=False)['vol change']))
#plt.show()

7283.50750493689
45


,Glucose,Na2HPO4,NaCl,KH2PO4,(NH4)2SO4,MgSO4,FeSO4,CaCl2,MnCl2,ZnSO4,CoCl2,CuSO4,Na2MoO4,Volume
Well,,,,,,,,,,,,,,
1,0.001759,0.040686,0.024663,0.011077,0.006003,0.001311,1.217626e-05,0.000066,1.023877e-05,7.211194e-07,3.881333e-06,7.552302e-07,1.386793e-07,1100.0
9,0.001759,0.040686,0.024663,0.011077,0.006003,0.001311,1.217626e-05,0.000066,1.023877e-05,7.211194e-07,3.881333e-06,7.552302e-07,1.386793e-07,1100.0
17,0.001759,0.040686,0.024663,0.011077,0.006003,0.001311,1.217626e-05,0.000066,1.023877e-05,7.211194e-07,3.881333e-06,7.552302e-07,1.386793e-07,1100.0
2,0.002972,0.053883,0.034610,0.017462,0.002430,0.002107,2.067163e-05,0.000080,2.527888e-06,8.725677e-06,2.718743e-06,3.741637e-07,5.158308e-07,1100.0
10,0.002972,0.053883,0.034610,0.017462,0.002430,0.002107,2.067163e-05,0.000080,2.527888e-06,8.725677e-06,2.718743e-06,3.741637e-07,5.158308e-07,1100.0
18,0.002972,0.053883,0.034610,0.017462,0.002430,0.002107,2.067163e-05,0.000080,2.527888e-06,8.725677e-06,2.718743e-06,3.741637e-07,5.158308e-07,1100.0
3,0.000499,0.016082,0.014708,0.003456,0.016173,0.000155,1.018272e-05,0.000106,8.234466e-06,8.812467e-06,2.183941e-06,3.506699e-07,6.158038e-07,1100.0
11,0.000499,0.016082,0.014708,0.003456,0.016173,0.000155,1.018272e-05,0.000106,8.234466e-06,8.812467e-06,2.183941e-06,3.506699e-07,6.158038e-07,1100.0
19,0.000499,0.016082,0.014708,0.003456,0.016173,0.000155,1.018272e-05,0.000106,8.234466e-06,8.812467e-06,2.183941e-06,3.506699e-07,6.158038e-07,1100.0


,srcpos,srcwell,destpos,destwell,vol
793,mixing_plate,33,dest_plate,42,8.000000
775,mixing_plate,33,dest_plate,34,8.000000
749,mixing_plate,25,mixing_plate,33,57.342627
750,water_plate,35,mixing_plate,33,180.000000
751,water_plate,35,mixing_plate,33,180.000000
752,water_plate,35,mixing_plate,33,180.000000
753,water_plate,36,mixing_plate,33,180.000000
754,water_plate,36,mixing_plate,33,180.000000
755,water_plate,36,mixing_plate,33,180.000000
756,water_plate,28,mixing_plate,33,62.657373


<Figure size 432x288 with 0 Axes>

<Figure size 432x288 with 0 Axes>

In [7]:
#Generate Tip Report
import math
print('Tips Needed By Subrutine')
operations = ['Adding Water To Mixing Plate','Adding Water to Destination Plate','Diluting Stock Solutions','Adding Undilute Media To Dest','Mixing Final Media']
for df,op in zip([water_mix_df,water_dest_df,mix_df,src_df,dest_df],operations):
    boxes = math.ceil(len(df)/96)
    print('{}: {} Tips,  {} Plates'.format(op,len(df),boxes))

total_tips = 16 + len(mix_df) + len(dest_df)
total_tips = len(biomek.transfer_df)
print('')
print('Overall Experiment Need')
print('Total Tips Needed:',total_tips)
print('Tip Plates Consumed:',math.ceil(total_tips/96))

Tips Needed By Subrutine
Adding Water To Mixing Plate: 210 Tips,  3 Plates
Adding Water to Destination Plate: 228 Tips,  3 Plates
Diluting Stock Solutions: 78 Tips,  1 Plates
Adding Undilute Media To Dest: 207 Tips,  3 Plates
Mixing Final Media: 380 Tips,  4 Plates

Overall Experiment Need
Total Tips Needed: 1103
Tip Plates Consumed: 12


In [8]:
#Figure out how to select for solute well
#solute == 'Glucose'
#REAGENT_WELL = (biomek.deck_df.loc[:,~biomek.deck_df.columns.isin([solute,'Volume'])] == 0).all(1)
#CONTAINS_SOLUTE = (biomek.deck_df.loc[:,biomek.deck_df.columns == solute] > 0).any(1)
#available_moles = biomek.deck_df.loc[REAGENT_WELL & CONTAINS_SOLUTE]
#available_moles[solute] = ((available_moles['Volume'] - dead_volume)/available_moles['Volume']) * available_moles[solute].loc[:,available_moles.columns != 'Volume']
#display(available_moles[solute])
